#  <span style="color:orange">Pycaret usando a base de dados do projeto</span>

# Carregando os dados

In [1]:
import pandas as pd
df = pd.read_feather('credit_scoring.ftr')

In [2]:
df.shape

(750000, 15)

In [3]:
df.columns

Index(['data_ref', 'index', 'sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'renda', 'mau'],
      dtype='object')

In [4]:
dataset = df.sample(50000)

In [5]:
dataset.drop(['data_ref','index'], axis=1, inplace=True)

In [6]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Conjunto de dados para modelagem (treino e teste): ' + str(data.shape))
print('Conjunto de dados não usados no treino/teste, apenas como validação: ' + str(data_unseen.shape))

Conjunto de dados para modelagem (treino e teste): (47500, 13)
Conjunto de dados não usados no treino/teste, apenas como validação: (2500, 13)


In [7]:
from pycaret.classification import *

# Configurando o pycaret

In [8]:
from pycaret.utils.generic import check_metric 

In [9]:
exp_clf101 = setup(data = data, target = 'mau', session_id=123) 

AttributeError: module 'pandas.core.strings' has no attribute 'StringMethods'

In [ ]:
data.dtypes

In [ ]:
#forçando a variável qnt de filhos como numérica
data.qtd_filhos = data.qtd_filhos.astype(float)

In [ ]:
exp_clf101 = setup(data = data, target = 'mau', session_id=123) 

# Comparando todos os modelos

In [ ]:
best_model = compare_models(fold=10)

In [ ]:
#ordenando a tabela por AUC
best_model = compare_models(fold=10, sort='AUC')

In [ ]:
print(best_model)

# Criando um modelo

In [ ]:
models()

### 8.3 Gradient Boosting Classifier

In [ ]:
gbc = create_model('gbc')

# Model tunning (Hyperparameter Tunning)

### Gradient Boosting

In [ ]:
tuned_gbc = tune_model(gbc, optimize='F1')

# Analisando o resultado do modelo

### 10.1 AUC Plot

In [ ]:
plot_model(tuned_gbc, plot = 'auc')

### 10.2 Precision-Recall Plot

In [ ]:
plot_model(tuned_gbc, plot = 'pr')

### 10.3 Importância das variáveis (Feature Importance) Plot

In [ ]:
plot_model(tuned_gbc, plot='feature')

### 10.4 Matriz de confusão (Confusion matrix)

In [ ]:
plot_model(tuned_gbc, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_gbc)

# Predict na amostra de validação / hold-out

In [ ]:
predict_model(tuned_gbc);

# Finalizando o modelo para deploy

In [ ]:
final_gbc = finalize_model(tuned_gbc)

In [ ]:
#Parâmetros finais do modelo Random Forest para deploy
print(final_gbc)

In [ ]:
predict_model(final_gbc);

# Predizendo o conjunto de dados que não foi usado no treino

In [ ]:
unseen_predictions = predict_model(final_gbc, data=data_unseen)
unseen_predictions.head()

In [ ]:
unseen_predictions.dtypes

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['mau'].astype(str), unseen_predictions['Label'], metric = 'Accuracy')

# Salvando o modelo final

In [ ]:
save_model(final_gbc,'Final GBC Model 02Jun2022')

(DICA: É sempre bom usar data no nome do arquivo ao salvar modelos, é bom para controle de versão.)

# Carregando o modelo salvo

Para carregar um modelo salvo em uma data futura no mesmo ambiente ou em um ambiente alternativo, usaríamos a função `load_model()` do PyCaret e, em seguida, aplicaríamos facilmente o modelo salvo em novos dados não vistos para previsão.

In [ ]:
saved_final_gbc = load_model('Final GBC Model 02Jun2022')

Uma vez que o modelo é carregado no ambiente, você pode simplesmente usá-lo para prever quaisquer novos dados usando a mesma função `predict_model()`. Abaixo, aplicamos o modelo carregado para prever o mesmo `data_unseen` que usamos na seção 13 acima.

In [ ]:
new_prediction = predict_model(saved_final_gbc, data=data_unseen)

In [ ]:
new_prediction.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(new_prediction['mau'].astype(str), new_prediction['Label'], metric = 'Accuracy')

In [ ]:
saved_final_gbc.named_steps